In [3]:
# pip install watermark
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn -g 
#,statsmodels

CPython 3.6.7
IPython 7.4.0

numpy 1.16.2
scipy 1.2.1
pandas 0.24.2
matplotlib 3.0.3
sklearn 0.20.3

compiler   : GCC 8.2.0
system     : Linux
release    : 4.9.93-boot2docker
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   :


In [36]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
#from glob import glob
import os
#import pickle
#pip install tqdm
#from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
#from scipy.sparse import csr_matrix
#import json
import math
#import seaborn as sns
#import matplotlib
#import matplotlib.pyplot as plt
#import matplotlib.ticker
#%matplotlib inline
import datetime
#from datetime import datetime, date, time,timedelta
import re

In [87]:
COMPARATOR_AUTHOR = "Anton Tushev" 
COMPARATOR_VERSION = "v.02"
PATH_TO_DATA = '/notebooks/pinda/data'
PATH_TO_STRUCG = '/notebooks/pinda/data/alcanc26_03_2019'
FILE_NAME_DATA1 = 'TX04.csv'
#FILE_NAME_DATA1 = 'test.csv'
IGNORE_CHART = ['@SIMD']
SKIP_EQUAL = True
COMPARATOR_README = "===================================================================================\r\n\
Comparator TDC <=> SimadynD software version: %s \r\nAuthor: %s \r\nComparation result made from %s \
file \r\nDate and time: %s \r\nUse filters: \r\nXXXXXXXX - critical points \
\r\n!!!!!!!! - not critical errors \r\nFalse - difference in programs\r\nTrue - no difference in programs \
\r\n===================================================================================\r\n\
"%(COMPARATOR_VERSION,COMPARATOR_AUTHOR,FILE_NAME_DATA1,datetime.datetime.now())
#print(COMPARATOR_README)


In [88]:
data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_DATA1),keep_default_na=False,na_values=[''], encoding="cp1251",sep=';',header=None)
#assign columns
data_a1 = data_a1[[0,1,3,5,11,28,29]]
data_a1.columns = ['rack_cpu','chart','block','signal','value','type','out']
data_a1 = pd.concat([data_a1, data_a1['rack_cpu'].str.split("\\",expand = True)[[0,1]]], axis=1) 
data_a1.columns = ['rack_cpu','chart','block','signal','value','type','out','rack','cpu']
data_a1["out"]= data_a1["out"].replace(("IN","OUT"),(0,1)).astype('bool') 
data_a1 = data_a1[~data_a1['chart'].isin(IGNORE_CHART)]
data_a1["block"] = data_a1["block"].astype(str)
print(data_a1['type'].unique())
print(data_a1['chart'].unique())
#data_a1[(data_a1['chart']=="RACK_1")&(data_a1['block']=="VM1")]


['BOOL' 'REAL' 'SDTIME' 'WORD' 'INT' 'DINT' 'GLOBAL' 'STRING' 'DWORD'
 'BYTE']
['ENCC_1' 'ENBRU1' 'ENBRI1' 'ENBEA1' 'BGT__1' '@SEND1' 'ENGAU1' 'ENGN11'
 'ENMB_1' 'ENOSW1' 'ENGUA1' 'ENHDR1' 'ENIHW1' 'ENINS1' 'ENEXT1' 'ENENT1'
 'ENSR_1' 'ENSPH1' 'ENSCD1' 'ENPIN1' 'ENPEL1' 'ENPAR1' 'ENISW1' 'ENLAP1'
 'ENMA_1' 'ENMAN1' 'ENOHW1' 'ENSTR1' 'ENTEM1' 'ENTUR1' 'MESY_1' 'MONIM1'
 'MONIW1' 'SERV_1' 'VISU_1' 'WPS__1' 'ENISW_MSU' 'ENDIA1' 'ENCRA1'
 'ENCPT1' 'ENCPS1' 'ENCPP1' 'ENCPM1' 'ENCPH1' 'ENCPC1' 'ENCPA1' 'ENCOO1'
 'ENMG_1' 'ENMC_1' 'ENMD_1' 'ENME_1' 'ENMF_1' '@SEND2' 'EXBAN2' 'EXBBH2'
 'EXBBY2' 'EXBEA2' 'EXBW_2' 'EXBWA2' 'EXBWT2' 'EXCC_2' 'EXCRA2' 'EXCRM2'
 'EXCUT2' 'EXDEF2' 'EXDIA2' 'EXFEL2' 'EXGAU2' 'EXGN12' 'EXGN22' 'EXIHW2'
 'EXIR_2' 'EXIRP2' 'EXISW2' 'EXKNA2' 'EXLAP2' 'EXMAN2' 'EXMLR2' 'EXMLT2'
 'EXOHW2' 'EXOSW2' 'EXPAR2' 'EXPUS2' 'EXREJ2' 'EXREL2' 'EXSC_2' 'EXSCA2'
 'EXSCM2' 'EXSH12' 'EXSH22' 'EXSHJ2' 'EXSHT2' 'EXSML2' 'EXSR_2' 'EXSRC2'
 'EXSRE2' 'EXSSC2' 'EXSSR2' 'EXSTR2' 'EXTB_2' 'EXTB

In [89]:
def get_some_value(rack,chart,block,signal,signal_type,PATH_TO_STRUCG=PATH_TO_STRUCG): 
    try:
        with open(os.path.join(PATH_TO_STRUCG,rack.lower(),chart.lower()+'.ofp')) as f:
            lines = [line.rstrip('\n') for line in f]
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        #print('ERROR File not exists!')
        return -4
    
    b = re.compile(r"\d+ ?%s +:"%(block))
    sf = re.compile(r"^ *\d+\s+%s\s+"%(signal))
    s = re.compile(r'(([<-] ?)|(@TYP=..,))([^@"\',&]+)([,"\'&]|$)')
    s_string = re.compile(r'(([-])|(@TYP=..,)) +(\'.+\')([,"\&]|$)')
    el = re.compile(r'^ *\d+\s+[+]')
    test_signal_no_zero = re.compile(r"[a-zA-Z_][0-9]$")
    if(test_signal_no_zero.search(signal)):
        signal_no_zero_check = True
        signal2 = signal[:-1]+"0"+signal[-1:]
        sf2 = re.compile(r"^ *\d+\s+%s\s+"%(signal2))
    else:
        signal_no_zero_check = False
    mark_search_signal = False
    mark_block_found = False
    mark_signal_found = False
    for line in lines:
        if block in line:
            if b.search(line):
                mark_search_signal = True
                mark_block_found = True
                #print("Block found >>> ", line)
                continue
        else:
            if mark_search_signal:
                if signal_no_zero_check:
                    cnd = bool(sf.search(line) or sf2.search(line))
                else:
                    cnd = bool(sf.search(line))
                if cnd:
                    mark_search_signal = False
                    #print("Signal found >>> ", line)
                    if signal_type == "STRING":
                        m=s_string.search(line)
                    else:
                        m=s.search(line)
                    #print(m)
                    #print(m.group(4))
                    if m:
                        #return normal result
                        return m.group(4)
                    else:
                        #print('ERROR Signal can not be recognized!')
                        return -3
                elif el.search(line):
                    return -2
                    
    if not mark_block_found:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!ERROR No block found!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return -1
    if mark_search_signal:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!ERROR No signal found!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return -2
    return -1
#@TYP=V2
#1219	X1  N2 < @TYP=O2,5,'Manual'
#value = get_some_value("TX03","EMUL_1","YS__03","X0")
#print("Comparison:      ",j['value'], ">>>",value)

In [242]:

tst_line = "$BLOC2Q"
#s = re.compile(r'((< ?)|(@TYP=..,))([^@"\',&]+)([,"\'&]|$)')
#s = re.compile(r"^ *\d+\s+A9\s+")
con_check = re.compile(r'(\S+[.]\D+)|(^\$\S+)')
m=con_check.search(tst_line)
print(m)
#print(m.group(4))


<_sre.SRE_Match object; span=(0, 7), match='$BLOC2Q'>


-2

In [92]:
tst_line = " 738	T1  NS - '@I@ HIGH PRESS. AGC. SYSTEM READY          '"
s = re.compile(r'(([-])|(@TYP=..,)) +(\'.+\')([,"\&]|$)')
m=s.search(tst_line)
print(m)
print(m.group(4))
val_sim = "5.0E2 [ms]"
units = 1000
tmp_txt = re.sub('[^0-9.E-]','', val_sim)
v_s = int(math.floor(float(tmp_txt)*units))
print(v_s)

<_sre.SRE_Match object; span=(12, 59), match="- '@I@ HIGH PRESS. AGC. SYSTEM READY          '">
'@I@ HIGH PRESS. AGC. SYSTEM READY          '
500000


In [92]:
def round_half_down(n, decimals=5):
    multiplier = 10 ** decimals
    return math.ceil(n*multiplier - 0.5) / multiplier

def twos_complement(hexstr,bits):
    value = int(hexstr,16)
    if value & (1 << (bits-1)):
        value -= 1 << bits
    return value

def toDeci(str,base): 
    def val(c): 
        if c >= '0' and c <= '9': 
            return ord(c) - ord('0') 
        else: 
            return ord(c) - ord('A') + 10; 
    llen = len(str) 
    power = 1 #Initialize power of base 
    num = 0     #Initialize result 
  
    # Decimal equivalent is str[len-1]*1 +  
    # str[len-1]*base + str[len-1]*(base^2) + ...  
    for i in range(llen - 1, -1, -1): 
          
        # A digit in input number must  
        # be less than number's base  
        if val(str[i]) >= base: 
            print('Invalid Number') 
            return -1
        num += val(str[i]) * power 
        power = power * base 
    return num 

def compare_values(val_tdc,val_sim,val_type):
    #['BOOL' 'INT' 'WORD' 'SDTIME' 'GLOBAL' 'STRING' 'REAL' 'DINT' 'BYTE' 'DWORD']
    #convert values first
    #print("val_sim:",val_sim)
    #print("val_type:",val_type)
    #check form of function output
    con_check = re.compile(r'(\S+[.]\D+)|(^\$\S+)')
    if con_check.search(val_sim):
        #print("Signal value of simadyn has chart connection")
        return -1
    if val_type=='REAL':
        #print(val_tdc)
        v_t = round_half_down(float(val_tdc.replace(',','.')))
        
        units = 1
        if "[" in val_sim:
            un_txt = val_sim.lower()
            un_txt = un_txt.split("[")[1]
            un_txt = un_txt.split("]")[0]
            if un_txt=="s":
                units = 1000
            elif un_txt=="min":
                units = 60000
            elif un_txt=="ms":
                units = 1  
            val_sim = val_sim.split("[")[0]
        if '%' in val_sim:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = round_half_down(float(tmp_txt)*units*0.01)
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = round_half_down(float(tmp_txt)*units)
    elif val_type=='BOOL':
        tmp_txt = val_sim.strip()
        if re.match(r'[10]$', tmp_txt):
            v_s = bool(val_sim)
        v_t = bool(val_tdc)
        
    elif val_type=='SDTIME': 
        v_t = int(val_tdc.strip('ms'))
        units = 1000
        un_txt = val_sim.lower()
        un_txt = un_txt.split("[")[1]
        un_txt = un_txt.split("]")[0]
        if un_txt=="s":
            units = 1000
        elif un_txt=="min":
            units = 60000
        elif un_txt=="ms":
            units = 1
        tmp_txt = re.sub('[^0-9.E-]','', val_sim)
        v_s = int(math.floor(round_half_down(float(tmp_txt)*units,decimals=3)))
    elif val_type=='INT' or val_type=='DINT':
        v_t = int(val_tdc)
        fl_check = re.compile(r'\d+[.]\d+')
        if fl_check.search(val_sim):
            #print("Signal in Simadyn has REAL instead INT")
            return -2
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            #v_s = toDeci(tmp_txt.strip(),16)
            v_s = twos_complement(tmp_txt.strip(),16)
        elif '%' in val_sim:
            tmp_txt = val_sim.replace('%','')
            tmp_txt = re.sub('[^0-9.E-]','', tmp_txt)
            v_s = int(tmp_txt)
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = int(val_sim)

    elif val_type=='WORD':
        tmp_txt = val_tdc.split('16#')[1]
        v_t = toDeci(tmp_txt.strip(),16)
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            v_s = toDeci(tmp_txt.strip(),16) 
        elif "B" in val_sim:
            tmp_txt = val_sim.split('0B')[1]
            tmp_txt =  tmp_txt.replace(" ", "")
            v_s = toDeci(tmp_txt.strip(),2)
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = int(tmp_txt)
    elif val_type=='BYTE':
        tmp_txt = val_tdc.split('16#')[1]
        v_t = toDeci(tmp_txt.strip(),16)
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            v_s = toDeci(tmp_txt.strip(),16) 
        elif "B" in val_sim:
            tmp_txt = val_sim.split('0B')[1]
            tmp_txt =  tmp_txt.replace(" ", "")
            v_s = toDeci(tmp_txt.strip(),2)
    elif val_type=='STRING':
        v_t = val_tdc.strip("'")
        v_s = val_sim.strip("'")
    #print("TDC: ",v_t)
    #print("SIM: ",v_s)
    return v_t==v_s




In [94]:
with open(FILE_NAME_DATA1.split(".")[0]+".txt", 'w') as f:
    print(COMPARATOR_README,file=f)
    data_a1_1 = data_a1
    for i, j in data_a1_1.iterrows():
        if(not j['out']) and not pd.isnull(j['value']):
            #print(j['rack'],j['chart'],j['block'],j['signal'])
            value = get_some_value(j['rack'],j['chart'],j['block'],j['signal'],j['type'])
            if value == -1:
                print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!BLOCK is not found in SimadynD software!!!!!!!!!!\r\n",file=f)
            elif value == -2:
                print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!\r\n",file=f)
            elif value == -3:
                print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL can not be recognized!!!!!!!!!!\r\n",file=f)
            elif value == -4:
                print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!FILE not exist!!!!!!!!!!\r\n",file=f)
            else:
                #try:
                compare_result = compare_values(j['value'],value,j['type'])
                if compare_result==-1:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL missing connection    XXXXXXXXXXXXXXXXXXXXXXXXXXX\r\n",file=f)
                elif compare_result==-2:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX\r\n",file=f)
                else:
                    if not compare_result or not SKIP_EQUAL:
                        print(j['rack'],j['chart'],j['block'],j['signal'], "Comparison:",j['value'], ">>>",value,">>>",compare_result,"\r\n",file=f)  
                    
                #except ValueError:
                #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably float in INT field XXXXXXXXXXXXXXXXXXXXXXXXXXX")
                #except UnboundLocalError:
                #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably lost chart connection XXXXXXXXXXXXXXXXXXXXXXXX")       

In [169]:
#data_a1[(data_a1['chart']=="RACK_1")&(data_a1['block']=="VM1")]

In [93]:

print(COMPARATOR_README)
data_a1_1 = data_a1
for i, j in data_a1_1.iterrows():
    if(not j['out']) and not pd.isnull(j['value']):
        #print(j['rack'],j['chart'],j['block'],j['signal'])
        value = get_some_value(j['rack'],j['chart'],j['block'],j['signal'],j['type'])
        if value == -1:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!")
        elif value == -2:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!")
        elif value == -3:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL can not be recognized!!!!!!!!!!")
        elif value == -4:
            print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!FILE not exist!!!!!!!!!!")
        else:
            #try:
            compare_result = compare_values(j['value'],value,j['type'])
            if compare_result==-1:
                print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL missing connection    XXXXXXXXXXXXXXXXXXXXXXXXXXX")
            elif compare_result==-2:
                print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX")
            else:
                if not compare_result or not SKIP_EQUAL:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "Comparison:",j['value'], ">>>",value,">>>",compare_result,"")  

            #except ValueError:
            #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably float in INT field XXXXXXXXXXXXXXXXXXXXXXXXXXX")
            #except UnboundLocalError:
            #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably lost chart connection XXXXXXXXXXXXXXXXXXXXXXXX")       

Comparator TDC <=> SimadynD software version: v.02 
Author: Anton Tushev 
Comparation result made from TX04.csv file 
Date and time: 2019-05-09 19:49:09.052460 
Use filters: 
XXXXXXXX - critical points 
!!!!!!!! - not critical errors 
False - difference in programs
True - no difference in programs 

TX04 ENCC_1 BLK10 T Comparison: 1000ms >>> 0.5[S] >>> False 
TX04 ENCC_1 CC_22 NF !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENCC_1 CC_24 NF !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENCC_1 CC_25 NF !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENCC_1 CC_26 NF !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENCC_1 CC_27 NF !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENCC_1 CC_28 NF !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENCC_1 CC_29 NF !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENCC_1 CONV1 NF !!!!!!!!!!SIGNAL is not found in SimadynD s

TX04 @SEND1 CP52_D21 N08 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N09 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N10 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N11 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N12 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N13 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N14 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N15 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N16 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N17 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 @SEND1 CP52_D21 N18 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!

TX04 ENOSW1 ENC1X1 AT !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X09 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X10 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X11 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X12 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X13 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X14 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X15 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENC1X1_0015_0022 X16 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENDMX1 AT !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENOSW1 ENDMX2_TRG X1 !!!!!!!!!!ERROR 

TX04 ENIHW1 RCV_DIA_D17X01_2 I10 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 RCV_DIA_D17X01_2 I11 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 RCV_DIA_D17X01_2 I12 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 RCV_DIA_D17X01_2 I13 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 RCV_DIA_D17X01_2 I14 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 RCV_DIA_D17X01_2 I15 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 RCV_DIA_D17X01_2 I16 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 SNUBBE OFS Comparison: -1,47E-02 >>> #-14.6624990E-3         >>> False 
TX04 ENIHW1 T3EN03 AR !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 T3EN03 TMX !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENIHW1 T3EN04 AR !!!!!!!!!!SIGNAL is not found in S

TX04 ENOHW1 ENM5 X13 XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX
TX04 ENOHW1 ENM5 X14 XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX
TX04 ENOHW1 ENM5 X15 XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX
TX04 ENOHW1 ENM5 X16 XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX
TX04 ENOHW1 ENM503 AT !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENOHW1 ENT303 AT !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENOHW1 ENT304 AT !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENOHW1 ENT305 AT !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENOHW1 ENT306 AT !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 ENOHW1 SND_DIA_D16X01_1 I1 !!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!
TX04 ENOHW1 SND_DIA_D16X01_1 I2 !!!!!!!!!!ERROR Block is not fo

...

TX04 EXDIA2 DIA076 T2 Comparison: '@D@88@ EX Rear Elevator Nao esta em Cima       ' >>> '@D@88@ EX Top Beam Back Sadle                  ' >>> False 
TX04 EXDIA2 DIA076 T3 Comparison: '@D@88@ EX Top Beam Back Sadle                  ' >>> '@D@88@ EX SPARE                                ' >>> False 
TX04 EXDIA2 DIA076 T4 Comparison: '@D@88@ EX Top Beam Back Sadle                  ' >>> '@D@88@ EX SPARE                                ' >>> False 
TX04 EXDIA2 DIA076 T17 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA076 T18 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA076 T19 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA076 T20 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA076 T21 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA076 T22 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA076 T23 !!!!!!!!!!SIGNAL is not fo

TX04 EXDIA2 DIA086 T17 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T18 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T19 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T20 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T21 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T22 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T23 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T24 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T25 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T26 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T27 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA086 T28 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!

TX04 EXDIA2 DIA102 T17 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T18 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T19 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T20 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T21 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T22 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T23 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T24 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T25 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T26 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T27 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA102 T28 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!

TX04 EXDIA2 DIA112 T4 Comparison: '@D@88@ EX CoilCar Nao esta naPosic/Opera Segura' >>> '@D@88@ EX SPARE                                ' >>> False 
TX04 EXDIA2 DIA112 T17 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T18 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T19 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T20 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T21 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T22 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T23 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T24 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T25 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2 DIA112 T26 !!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!
TX04 EXDIA2

KeyboardInterrupt: 